In [2]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 10.5 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=659b92616bb1fd0d323352d52a8cceb9e676833a8db68b2a2991a8cfc0ce7ee7
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [3]:
import os
import cv2
from google.colab import drive
from deepface import DeepFace
import pandas as pd

def generate_dataset():
  drive.mount('/gdrive', force_remount=True)
  data_folder = '/gdrive/MyDrive/face_dataset'
  images = []
  files = []
  feature = dict()
  featuress = []

  for f in os.listdir(data_folder) :
      files.append(f)

  for folder in files:
    new_path = os.path.join(data_folder, folder)
    for filename in os.listdir(new_path) :
      embedding_objs = DeepFace.represent(img_path = os.path.join(new_path, filename), model_name = "ArcFace", enforce_detection = False)
      feature['label'] = folder
      feature['file_name'] = filename
      for i in range(len(embedding_objs[0]["embedding"])) :
        feature[f'feature{i+1}'] = embedding_objs[0]["embedding"][i]
      feature_copy = feature.copy()
      featuress.append(feature_copy)
      feature.clear()

  df_input = pd.DataFrame(featuress)
  df_input.to_csv('/gdrive/MyDrive/dataset/input_data.csv', mode = 'a', index = False)


# generate_dataset()


Directory  /root /.deepface created
Directory  /root /.deepface/weights created


In [4]:
drive.mount('/gdrive', force_remount=True)
df = pd.read_csv('/gdrive/MyDrive/dataset/input_data.csv')
df.head()

Mounted at /gdrive


,label,file_name,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,...,feature503,feature504,feature505,feature506,feature507,feature508,feature509,feature510,feature511,feature512
0,Taraneh_Alidoosti,large_c3102545-a6e2-46b8-b1c8-3229fc257b83-0.jpg,0.131124,0.240982,0.031335,-0.140632,-0.085016,0.032476,-0.117792,-0.008110,...,-0.199647,-0.055599,-0.117984,-0.283036,0.024422,0.046395,-0.235830,0.259740,-0.006825,0.058445
1,Taraneh_Alidoosti,n00500142-b-4.jpg,0.110742,0.513270,-0.174584,0.066744,-0.282779,0.010204,-0.087545,-0.313360,...,-0.282334,-0.025950,0.054783,-0.133751,-0.083157,-0.146127,-0.239894,0.534455,-0.106297,-0.013161
2,Taraneh_Alidoosti,maxresdefault (2)-0.jpg,-0.015313,0.317469,-0.241807,-0.124789,-0.077396,-0.052253,-0.043101,-0.131162,...,-0.296250,0.018286,0.031260,-0.096230,0.117513,-0.041375,0.079029,0.388054,0.006770,-0.086044
3,Taraneh_Alidoosti,parinaz-1-1.jpg,-0.288929,0.887867,-0.200956,-0.464644,-0.066016,0.576108,-0.248666,-0.228823,...,-1.453093,0.247549,0.606108,-0.970516,0.405728,-0.325398,-0.125744,-0.071751,-0.486460,-1.563897
4,Taraneh_Alidoosti,ali_daie_2-0.jpg,0.110708,0.362302,-0.351462,-0.263411,-0.047563,-0.232761,0.013197,0.079611,...,-0.089996,0.173191,0.039924,-0.228698,0.013477,0.039058,-0.168751,0.130101,-0.107311,-0.066673


In [38]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

df=df.dropna()
X = df.drop(['label','file_name'],axis=1)
y = df['label']
y=y.to_frame()
print(y.head)
enc = OneHotEncoder()
enc.fit(y)
onehotlabels = enc.transform(y).toarray()
labels = pd.DataFrame(onehotlabels)
X_train, X_test, Y_train, Y_test = train_test_split(X,labels,test_size=0.1)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape
drive.mount('/gdrive', force_remount=True)
labels.to_csv('/gdrive/MyDrive/dataset/label.csv', mode = 'a', index = False)
labels.head

<bound method NDFrame.head of                   label
0     Taraneh_Alidoosti
1     Taraneh_Alidoosti
2     Taraneh_Alidoosti
3     Taraneh_Alidoosti
4     Taraneh_Alidoosti
...                 ...
1722        Behnam_Bani
1723        Behnam_Bani
1724        Behnam_Bani
1725        Behnam_Bani
1726        Behnam_Bani

[1727 rows x 1 columns]>
Mounted at /gdrive


<bound method NDFrame.head of        0    1    2    3    4    5    6    7    8    9   ...   20   21   22  \
0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
1722  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1723  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1724  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1725  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1726  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

       23   24   25   26   27   2

In [26]:
model = tf.keras.models.Sequential([
tf.keras.layers.Dense(512, activation='relu'),
tf.keras.layers.Dense(1000, activation='relu'),
tf.keras.layers.Dense(256, activation='sigmoid'),
tf.keras.layers.Dense(30, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=100)
model.evaluate(X_test, Y_test)
drive.mount('/gdrive', force_remount=True)
model.save("/gdrive/MyDrive/dataset/face_model.h5")

Epoch 1/100
49/49 [==============================] - 6s 8ms/step - loss: 0.2056 - accuracy: 0.0547
Epoch 2/100
49/49 [==============================] - 0s 6ms/step - loss: 0.1383 - accuracy: 0.1628
Epoch 3/100
49/49 [==============================] - 0s 6ms/step - loss: 0.1277 - accuracy: 0.3912
Epoch 4/100
49/49 [==============================] - 0s 8ms/step - loss: 0.1174 - accuracy: 0.5097
Epoch 5/100
49/49 [==============================] - 0s 7ms/step - loss: 0.1068 - accuracy: 0.6075
Epoch 6/100
49/49 [==============================] - 0s 6ms/step - loss: 0.0952 - accuracy: 0.7079
Epoch 7/100
49/49 [==============================] - 1s 18ms/step - loss: 0.0828 - accuracy: 0.7716
Epoch 8/100
49/49 [==============================] - 0s 6ms/step - loss: 0.0724 - accuracy: 0.7921
Epoch 9/100
49/49 [==============================] - 0s 5ms/step - loss: 0.0637 - accuracy: 0.8012
Epoch 10/100
49/49 [==============================] - 0s 7ms/step - loss: 0.0560 - accuracy: 0.8089
Epoch 11

In [7]:
type(X_train)

pandas.core.frame.DataFrame

In [42]:
import os
import cv2
from google.colab import drive
from deepface import DeepFace
import pandas as pd

drive.mount('/gdrive', force_remount=True)
data_folder = '/gdrive/MyDrive/face_dataset'
images = []
files = []
feature = dict()
featuress = []

for f in os.listdir(data_folder) :
    files.append(f)

files.sort()
print(files)

Mounted at /gdrive
['Adel_FerdowsiPour', 'Ali_Daei', 'Ali_Khamenei', 'Asghar_Farhadi', 'Bahare_Rahnama', 'Bahram_Radan', 'Behnam_Bani', 'Dariush_Arjmand', 'Ebi', 'Ehsan_Alikhani', 'Elham_Hamidi', 'Elnaz_Shakerdoost', 'Golshifteh_Farahani', 'Hamid_Lolaei', 'Hootan_Shakiba', 'Javad_Khiabani', 'Javad_Razavian', 'Leyla_Hatami', 'Mahnaz_Afshar', 'Mehran_Ghafourian', 'Mehran_Modiri', 'Mohsen_Chavoshi', 'Parinaz_Izadyar', 'Parsa_Pirozfar', 'Parviz_Parastooee', 'Shahab_Hosseini', 'Siamak_Ansari', 'Siavash_Ghomayshi', 'Tannaz_Tabatabaee', 'Taraneh_Alidoosti']


In [44]:
import numpy as np

embedding_objs = DeepFace.represent(img_path = '/content/images (1).jpg', model_name = "ArcFace")

a=np.asarray(embedding_objs[0]["embedding"])
a = a.reshape(-1,512)
print(a.shape)
print(len(embedding_objs[0]["embedding"]))
out = model.predict(a)
out = np.argmax(out)
print(out)
files[out]

(1, 512)
512
1/1 [==============================] - 0s 21ms/step
6


'Behnam_Bani'